## If you are using Colab for free, we highly recommend you activate the T4 GPUhardware accelerator. Our models are designed to run with at least 16GB of RAM, activating T4 will grant the notebook 16GB of GDDR6 RAM as opposed to the ~13GB Colab gives automatically.

## To activate T4:
1. click on the "Runtime" tab
2. click on "Change runtime type"
3. select T4 GPU under "Hardware Accelerator"

### NOTE: there is a weekly usage limit on using T4 for free

# Notebook Example : Email Organizer using LLMware

This notebook demonstrates a robust email organizer built with LLMWare, designed to handle diverse email types and intelligently classify them based on urgency, intent and sentiment. The model processes email content to identify high-priority items, categorize intent, and assess sentiment, enhancing productivity by prioritizing important messages and filtering out low-urgency emails.

In [1]:
# Install the required package
!pip install llmware -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 6.9 MB/s eta 0:00:00


In [2]:
# Import necessary modules from LLMware
from llmware.agents import LLMfx
import pandas as pd


In [3]:
#  sample email dataset
emails = [
    {
        "sender": "ceo@company.com",
        "subject": "Quarterly Financial Results",
        "content": "The company has experienced significant growth in revenue...",
        "received_time": "2024-10-29 10:30"
    },
    {
        "sender": "hr@company.com",
        "subject": "Urgent: Complete Your Annual Review",
        "content": "Please complete your annual review by the end of the month...",
        "received_time": "2024-10-29 12:45"
    },
    {
        "sender": "support@bank.com",
        "subject": "Action Required: Unusual Login Activity",
        "content": "We noticed a new login attempt from an unrecognized device. Please verify your identity.",
        "received_time": "2024-10-29 08:15"
    },
    {
        "sender": "newsletter@shopping.com",
        "subject": "New Offers for You!",
        "content": "Check out our latest promotions on electronics and apparel.",
        "received_time": "2024-10-28 09:30"
    },
    {
        "sender": "project_manager@company.com",
        "subject": "Weekly Project Update",
        "content": "Here is the weekly update on the project status. Please review and provide feedback.",
        "received_time": "2024-10-27 14:00"
    },
    {
        "sender": "legal@company.com",
        "subject": "Important: New Compliance Policy",
        "content": "All employees are required to read and acknowledge the new compliance policy...",
        "received_time": "2024-10-26 16:20"
    },
    {
        "sender": "social@network.com",
        "subject": "Your friend mentioned you in a comment!",
        "content": "You have been tagged in a comment. See what they said.",
        "received_time": "2024-10-25 11:10"
    },
]

In [4]:
# Function to process and classify emails robustly
def organize_emails(emails):
    """
    Organize emails into categories based on urgency, sender, and content type.
    """

    agent = LLMfx(verbose=True)
    agent.load_tool("sentiment")
    agent.load_tool("intent")

    organized_emails = []

    for email in emails:
        # Preprocess the content for classification
        content = f"{email['subject']} {email['content']}"

        try:
            # Sentiment analysis
            sentiment_response = agent.sentiment(content)
            sentiment = sentiment_response.get("llm_response", {}).get("sentiment", ["neutral"])[0]  # Get string sentiment
            confidence = sentiment_response.get("confidence_score", 0.0)

            # Intent classification to determine type (work, personal, promotional, etc.)
            intent_response = agent.intent(content)
            intent = intent_response.get("llm_response", {}).get("intent", ["general"])[0]  # Get string intent

            # Determine urgency based on keywords and sentiment confidence
            urgency = "High" if "urgent" in email['subject'].lower() or confidence > 0.85 else "Normal"

            # Construct categorized email information
            categorized_email = {
                "Sender": email["sender"],
                "Subject": email["subject"],
                "Content": email["content"],
                "Urgency": urgency,
                "Sentiment": sentiment,
                "Intent": intent,
                "Received Time": email["received_time"]
            }

            organized_emails.append(categorized_email)

        except Exception as e:
            print(f"Error processing email from {email['sender']}: {e}")

    return organized_emails



In [5]:
# Run the email organizer
organized_emails = organize_emails(emails)

INFO:llmware.agents:Agent - Setting up LLMWare Workspace.


update: Launching LLMfx process
step - 	1 - 	creating object - ready to start processing.
step - 	2 - 	loading tool - sentiment


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1212: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/12.0k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/5.77k [00:00<?, ?B/s]

slim-intent.gguf:   0%|          | 0.00/669M [00:00<?, ?B/s]

step - 	4 - 	executing function call - deploying - sentiment 
step - 	5 - 	loading new processing text - 1 new entries
step - 	6 - 	executing function call - getting response - sentiment
				 -- llm_response - {'sentiment': ['positive']}
				 -- output type - dict
				 -- usage - {'input': 35, 'output': 9, 'total': 44, 'metric': 'tokens', 'processing_time': 0.7747218608856201, 'type': 'dict'}


tokenizer_tl.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

step - 	7 - 	analyzing response - sentiment
				 -- confidence score - 0.994
				 -- analyzing response - sentiment': ['positive']}
				 -- analyzing response - {'choice_1': ['pos', 0.994, 1066], 'choice_2': ['neut', 0.005, 17821], 'choice_3': ['negative', 0.001, 22198]}
				 -- analyzing response - [('positive', 1066, 0.994), ('neutral', 17821, 0.005), ('negative', 22198, 0.001)]
step - 	8 - 	executing function call - deploying - intent 
step - 	9 - 	loading new processing text - 1 new entries
step - 	10 - 	executing function call - getting response - intent
				 -- llm_response - {'intent': ['subscription']}
				 -- output type - dict
				 -- usage - {'input': 35, 'output': 8, 'total': 43, 'metric': 'tokens', 'processing_time': 0.3714628219604492, 'type': 'dict'}
step - 	11 - 	analyzing response - intent
				 -- confidence score - 0.431
				 -- analyzing response - intent': ['subscription']}
				 -- analyzing response - {'choice_1': ['sub', 0.431, 1491], 'choice_2': ['in', 0.155, 262

In [6]:
# Convert to DataFrame for easy viewing and sorting
df = pd.DataFrame(organized_emails)
df.sort_values(by=["Urgency", "Sentiment"], ascending=[False, True], inplace=True)
print(df)


                        Sender                                  Subject  \
2             support@bank.com  Action Required: Unusual Login Activity   
3      newsletter@shopping.com                      New Offers for You!   
4  project_manager@company.com                    Weekly Project Update   
5            legal@company.com         Important: New Compliance Policy   
6           social@network.com  Your friend mentioned you in a comment!   
1               hr@company.com      Urgent: Complete Your Annual Review   
0              ceo@company.com              Quarterly Financial Results   

                                             Content Urgency Sentiment  \
2  We noticed a new login attempt from an unrecog...  Normal  negative   
3  Check out our latest promotions on electronics...  Normal   neutral   
4  Here is the weekly update on the project statu...  Normal   neutral   
5  All employees are required to read and acknowl...  Normal   neutral   
6  You have been tagged in a 